In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings
import seaborn as sns
import plotly
import plotly.express as px

warnings.simplefilter(action='ignore')
mpl.rcParams['figure.dpi'] = 105
mpl.rcParams['figure.figsize'] = (7, 5)

In [7]:
#Set noticias
from sklearn.datasets import fetch_20newsgroups
data_20_news = fetch_20newsgroups()

In [15]:
#Parte 2: Pipelines y Gridsearch
from sklearn.datasets import fetch_20newsgroups
data_20_news = fetch_20newsgroups()

In [16]:
print(f"Samples: {len(data_20_news.data)}")
#print(f"Clases: {data_20_news.target_names}")
dictAx = {index: f"Clase:{index}, {feature}" for index, feature in enumerate(data_20_news.target_names)}
print(f"\nEstratificacion:\n{pd.Series(data_20_news.target).value_counts().sort_index().rename(index=dictAx)}")

Samples: 11314

Estratificacion:
Clase:0, alt.atheism                 480
Clase:1, comp.graphics               584
Clase:2, comp.os.ms-windows.misc     591
Clase:3, comp.sys.ibm.pc.hardware    590
Clase:4, comp.sys.mac.hardware       578
Clase:5, comp.windows.x              593
Clase:6, misc.forsale                585
Clase:7, rec.autos                   594
Clase:8, rec.motorcycles             598
Clase:9, rec.sport.baseball          597
Clase:10, rec.sport.hockey           600
Clase:11, sci.crypt                  595
Clase:12, sci.electronics            591
Clase:13, sci.med                    594
Clase:14, sci.space                  593
Clase:15, soc.religion.christian     599
Clase:16, talk.politics.guns         546
Clase:17, talk.politics.mideast      564
Clase:18, talk.politics.misc         465
Clase:19, talk.religion.misc         377
dtype: int64


In [17]:
# Se trabajará sólo con 2 categorias: 'alt.atheism', 'talk.religion.misc'
#Test

categories = ['alt.atheism', 'talk.religion.misc']
text_data = fetch_20newsgroups(subset ='train', 
                          categories = categories)

print(f"{len(text_data.filenames)} documentos y {len(text_data.target_names)} categorías.")
X_train = text_data.data
y_train = text_data.target

857 documentos y 2 categorías.


In [18]:
# Realizamos el mismo proceso para los datos de prueba.
text_data_test = fetch_20newsgroups(subset ='test', categories = categories)

print(f"{len(text_data_test.filenames)} documentos y {len(text_data_test.target_names)} categorías.")

X_test = text_data_test.data
y_test = text_data_test.target

570 documentos y 2 categorías.


In [19]:
#Revision de datos
print(X_train[400])
print(y_train[400])

From: V2110A@VM.TEMPLE.EDU (Richard Hoenes)
Subject: Re: A Message for you Mr. President: How do you know what happened?
Organization: Temple University
Lines: 31
Nntp-Posting-Host: vm.temple.edu
X-Newsreader: NNR/VM S_1.3.2

In article <cjkC5sy5G.Ko4@netcom.com>
cjk@netcom.com writes:
 
>This was obviously a lot different than the ordinary FBI adventure.
>
>I believe that the Federal officers had a conflict of interests here.
>
>Throught out the whole affair, it seamed to me that they were chiefly
>concerned with saving face rather than saving lifes.  Its true that
>The BD were resisting arrest and that they should have surrendered
>when they first realized that these where federal officers.  But they
>didn`t.
 
I'm not sure what you mean by 'saving face' unless you are confusing
the FBI with the BATF who are the ones who were in charge of the
original search warrant.
 
>But when they didn`t, the FBI should not have treated as a hostage
>situation, it wasn't.
>
>I think  more discussi

In [20]:
#Tokenizacion por medio de CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
      max_df = 0.5,            # Los tokens con proporción de aparición mayor que 'max_df' son ignorados.
      analyzer = 'word',       # Los tokens corresponderán a palabras (este es el valor por defecto).
      ngram_range = (1, 1),    # Limite inferior e superior de tokens por n-grama.
      max_features = 5000      # Tamaño máximo del vocabulario.
)
vector_text = vectorizer.fit_transform(X_train)

In [21]:
#revision preliminar transformacion por CountVectorizer
print(f"Tamaño: {vector_text.shape}\nTipo: {type(vector_text)}\nDiccionario: {len(vectorizer.vocabulary_)} Terminos")
print(vector_text[0])
#print(vectorizer.get_feature_names()) #Vector con features luego de tokenizar

Tamaño: (857, 5000)
Tipo: <class 'scipy.sparse.csr.csr_matrix'>
Diccionario: 5000 Terminos
  (0, 2766)	3
  (0, 1167)	2
  (0, 4654)	2
  (0, 854)	1
  (0, 4912)	2
  (0, 2879)	1
  (0, 51)	1
  (0, 591)	1
  (0, 3896)	1
  (0, 2005)	1
  (0, 491)	1
  (0, 1076)	4
  (0, 782)	1
  (0, 1663)	3
  (0, 3571)	1
  (0, 2572)	3
  (0, 1103)	1
  (0, 4496)	2
  (0, 1265)	1
  (0, 1424)	2
  (0, 4906)	1
  (0, 3138)	1
  (0, 1271)	1
  (0, 3695)	1
  (0, 4479)	2
  (0, 1311)	1
  (0, 4860)	1
  (0, 4760)	1
  (0, 955)	1
  (0, 4187)	1
  (0, 4886)	1
  (0, 1736)	1
  (0, 1722)	2
  (0, 1314)	2
  (0, 2863)	1
  (0, 3143)	1
  (0, 3780)	1
  (0, 3318)	3
  (0, 1974)	2
  (0, 3092)	1
  (0, 4301)	1
  (0, 928)	1
  (0, 4173)	1
  (0, 4980)	1
  (0, 718)	1
  (0, 3472)	1
  (0, 4495)	1
  (0, 4562)	1
  (0, 2783)	1
  (0, 2711)	1


In [22]:
#Features para elemento vector_text[0]
feature_names = vectorizer.get_feature_names()

for ind in vector_text[0].indices:  
    print(f'El token "{feature_names[ind]}" aparece {vector_text[0, ind]} {"vez" if vector_text[0, ind]==1 else "veces"}.')

El token "mangoe" aparece 3 veces.
El token "cs" aparece 2 veces.
El token "umd" aparece 2 veces.
El token "charley" aparece 1 vez.
El token "wingate" aparece 2 veces.
El token "metaphysics" aparece 1 vez.
El token "24" aparece 1 vez.
El token "benedikt" aparece 1 vez.
El token "rosenau" aparece 1 vez.
El token "great" aparece 1 vez.
El token "authority" aparece 1 vez.
El token "contradictory" aparece 4 veces.
El token "cannot" aparece 1 vez.
El token "exist" aparece 3 veces.
El token "property" aparece 1 vez.
El token "language" aparece 3 veces.
El token "correct" aparece 1 vez.
El token "things" aparece 2 veces.
El token "defined" aparece 1 vez.
El token "do" aparece 2 veces.
El token "will" aparece 1 vez.
El token "object" aparece 1 vez.
El token "definitions" aparece 1 vez.
El token "reality" aparece 1 vez.
El token "then" aparece 2 veces.
El token "described" aparece 1 vez.
El token "we" aparece 1 vez.
El token "ve" aparece 1 vez.
El token "come" aparece 1 vez.
El token "something

In [23]:
#Analisis de frecuencias Tf-Idf
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer(use_idf=True)
transformed_text = transformer.fit_transform(vector_text)

for ind in transformed_text[0].indices:  
    print(f'El token "{feature_names[ind]}" tiene un valor de tf-idf de: {transformed_text[0, ind]:5f}.') 

El token "yet" tiene un valor de tf-idf de: 0.066565.
El token "wingate" tiene un valor de tf-idf de: 0.192767.
El token "will" tiene un valor de tf-idf de: 0.046972.
El token "which" tiene un valor de tf-idf de: 0.044033.
El token "we" tiene un valor de tf-idf de: 0.040011.
El token "ve" tiene un valor de tf-idf de: 0.060770.
El token "umd" tiene un valor de tf-idf de: 0.181395.
El token "tove" tiene un valor de tf-idf de: 0.117556.
El token "things" tiene un valor de tf-idf de: 0.118716.
El token "thing" tiene un valor de tf-idf de: 0.061815.
El token "then" tiene un valor de tf-idf de: 0.093011.
El token "strife" tiene un valor de tf-idf de: 0.114117.
El token "something" tiene un valor de tf-idf de: 0.056534.
El token "sod" tiene un valor de tf-idf de: 0.117556.
El token "rosenau" tiene un valor de tf-idf de: 0.089084.
El token "remember" tiene un valor de tf-idf de: 0.081653.
El token "reality" tiene un valor de tf-idf de: 0.091262.
El token "property" tiene un valor de tf-idf de:

In [24]:
print(f"Set transformado: {transformed_text.shape}")

Set transformado: (857, 5000)


In [25]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(    
        n_estimators = 50,    # Número de árboles de decisión del árbol.
        max_features =  0.5   # Proporción máxima de características a considerar.
)

clf.fit(transformed_text, y_train)

RandomForestClassifier(max_features=0.5, n_estimators=50)

In [26]:
#Pipeline de ejecucion de todo el flujo de trabajo:
from sklearn.pipeline import Pipeline

pipeline = Pipeline(steps=[("countVectorizer", CountVectorizer()),
                           ("tfidf", TfidfTransformer()),
                           ("clfRandomForest", RandomForestClassifier())])

parameters = {      
    'countVectorizer__max_features': (1000, 2000),      # Tamaño máximo del vocabulario (1000 y 2000).
    'countVectorizer__ngram_range': ((1, 1), (1, 2)),   # Unigramas y unigramas + bigramas.
    'tfidf__use_idf': (True, False),         # Usar o no frecuencia inversa del documento.
    'clfRandomForest__n_estimators': [50, 100],          # Número de árboles del Random Forest (50 y 100).
    'clfRandomForest__max_features': [0.1, 0.3]          # Porcentaje de características del Random Forest (10% y 30%).
}

In [27]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(pipeline,   # En este caso el modelo entregado a GridSearch es un objeto PipeLine.
                           parameters,
                           scoring="f1",
                           cv=3)
                                       # de ejecución y evaluación en cada entrenamiento.

In [28]:
#Ejecucion de GridSearch con pipeline

grid_search.fit(text_data.data, text_data.target)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('countVectorizer', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clfRandomForest',
                                        RandomForestClassifier())]),
             param_grid={'clfRandomForest__max_features': [0.1, 0.3],
                         'clfRandomForest__n_estimators': [50, 100],
                         'countVectorizer__max_features': (1000, 2000),
                         'countVectorizer__ngram_range': ((1, 1), (1, 2)),
                         'tfidf__use_idf': (True, False)},
             scoring='f1')

In [29]:
df = pd.DataFrame(grid_search.cv_results_)
filter_col = [col for col in df if col.startswith('param')]
filter_col.append("rank_test_score")
df[filter_col].sort_values(by="rank_test_score")

,param_clfRandomForest__max_features,param_clfRandomForest__n_estimators,param_countVectorizer__max_features,param_countVectorizer__ngram_range,param_tfidf__use_idf,params,rank_test_score
12,0.1,100,2000,"(1, 1)",True,"{'clfRandomForest__max_features': 0.1, 'clfRan...",1
15,0.1,100,2000,"(1, 2)",False,"{'clfRandomForest__max_features': 0.1, 'clfRan...",2
20,0.3,50,2000,"(1, 1)",True,"{'clfRandomForest__max_features': 0.3, 'clfRan...",3
13,0.1,100,2000,"(1, 1)",False,"{'clfRandomForest__max_features': 0.1, 'clfRan...",4
6,0.1,50,2000,"(1, 2)",True,"{'clfRandomForest__max_features': 0.1, 'clfRan...",5
14,0.1,100,2000,"(1, 2)",True,"{'clfRandomForest__max_features': 0.1, 'clfRan...",6
5,0.1,50,2000,"(1, 1)",False,"{'clfRandomForest__max_features': 0.1, 'clfRan...",7
9,0.1,100,1000,"(1, 1)",False,"{'clfRandomForest__max_features': 0.1, 'clfRan...",8
8,0.1,100,1000,"(1, 1)",True,"{'clfRandomForest__max_features': 0.1, 'clfRan...",9
28,0.3,100,2000,"(1, 1)",True,"{'clfRandomForest__max_features': 0.3, 'clfRan...",10


In [30]:
print(grid_search.best_estimator_)
print(f"Score: {grid_search.best_score_}")
grid_search.best_params_

Pipeline(steps=[('countVectorizer', CountVectorizer(max_features=2000)),
                ('tfidf', TfidfTransformer()),
                ('clfRandomForest', RandomForestClassifier(max_features=0.1))])
Score: 0.9084688963311415


{'clfRandomForest__max_features': 0.1,
 'clfRandomForest__n_estimators': 100,
 'countVectorizer__max_features': 2000,
 'countVectorizer__ngram_range': (1, 1),
 'tfidf__use_idf': True}

In [31]:
pred = grid_search.predict(X_test)
print(f"Mejor puntaje (Train): {grid_search.best_score_:.3f}")

Mejor puntaje (Train): 0.908


In [32]:
from sklearn.metrics import confusion_matrix

pd.DataFrame(confusion_matrix(y_test, pred), columns=["a","b"], index=["a","b"])
print(f"Mejor puntaje (Test): {grid_search.score(X_test, y_test):.3f}")

Mejor puntaje (Test): 0.793


In [35]:
print(type(X_train))

<class 'list'>


In [33]:
data_20_news.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])